<a href="https://colab.research.google.com/github/lucianoigit/Data_scientist/blob/main/Chatairbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Autor : Luciano Iriarte
Tema: Clasificacion de departamentos disponibles en el dataset de Airbnb buenos aires
Objetivo: Chat interactivo para obtener una recomendacion de alquiler segun un una descripcion de entrada por el usuario.
fecha: 5/5/2023

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# Cargar los datos de Airbnb
df = pd.read_csv('/content/listings.csv.gz')
print(df.shape)
print(df.columns)


print(f"hay datos ausentes?{df.isnull().any().any()}")
# Analizamos el dataset 
df.info()
df.beds.head(10)



(20085, 75)
Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_a

0    1.0
1    1.0
2    2.0
3    1.0
4    1.0
5    1.0
6    1.0
7    2.0
8    1.0
9    2.0
Name: beds, dtype: float64

In [2]:
df=df.drop(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source',
       'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_group_cleansed', 'latitude',
       'longitude','bathrooms','minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],axis=1 )



In [3]:
print(df.columns)
df.info()



Index(['name', 'description', 'neighborhood_overview', 'neighbourhood',
       'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'review_scores_rating', 'instant_bookable'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20085 entries, 0 to 20084
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    20082 non-null  object 
 1   description             19566 non-null  object 
 2   neighborhood_overview   11420 non-null  object 
 3   neighbourhood           11420 non-null  object 
 4   neighbourhood_cleansed  20085 non-null  object 
 5   property_type           20085 non-null  object 
 6   room_type               20085 non-null  object 
 7   accommodates            20085 non-null  int64  
 8   b

In [4]:
df.name

0                  Amazing Luxurious Apt-Palermo Soho
1        RELAX IN HAPPY HOUSE - PALERMO, BUENOS AIRES
2               Amazing view - GREAT Studio - Balcony
3                         Single Room in Palermo Soho
4                               ROOM WITH RIVER SIGHT
                             ...                     
20080                 Hermoso departamento en Cañitas
20081                       Departamento de categoría
20082                  Loft, parrilla y terraza en BA
20083                   Dpto.un dormitorio con balcón
20084                   Departamento recoleta amplio.
Name: name, Length: 20085, dtype: object

In [5]:
import re
def preprocesar_texto(texto):
    if isinstance(texto, str):
        # eliminar etiquetas HTML
        texto = re.sub(r'<[^>]+>', '', texto)
        # eliminar caracteres que no sean texto o números
        texto = re.sub(r'\W', ' ', texto)
        # convertir a minúsculas
        texto = texto.lower()
    return texto




In [6]:
for columna in df.columns:
    df[columna] = df[columna].apply(preprocesar_texto)

TOMO LA DECISION PARA ESTE CASO DE UTILIZAR SOLO A COLUMNA DESCRIPTION Y ROOM_TYPE.

Mas adelante utilizaremos mas parametros para el aprendizaje

In [12]:
df = df[['description', 'name']]

In [15]:
df = df.dropna(subset=['description'])

In [16]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19564 entries, 0 to 20084
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  19564 non-null  object
 1   name         19564 non-null  object
dtypes: object(2)
memory usage: 458.5+ KB


,description,name
0,the spaceluxurious new apt 1 bdrm pool gym ...,amazing luxurious apt palermo soho
1,beautiful cozy apartment in excellent location...,relax in happy house palermo buenos aires
2,the apartment is in an upscale brand new build...,amazing view great studio balcony
3,the spacethis single room shared bathroom ha...,single room in palermo soho
4,the spacei offer a room in my apartment locate...,room with river sight
...,...,...
20080,relajate en este alojamiento único y tranquilo,hermoso departamento en cañitas
20081,inmejorable zona palier privado hermoso depart...,departamento de categoría
20082,disfrutá de la sencillez de este alojamiento t...,loft parrilla y terraza en ba
20083,hermoso depto de un dormitorio en belgrano id...,dpto un dormitorio con balcón


In [17]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    words = sentence.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['description'] = df['description'].apply(remove_stopwords)

<ipython-input-18-a81f55ef4123>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = df['description'].apply(remove_stopwords)


In [19]:
df

,description,name
0,spaceluxurious new apt 1 bdrm pool gym spa 24 ...,amazing luxurious apt palermo soho
1,beautiful cozy apartment excellent location pa...,relax in happy house palermo buenos aires
2,apartment upscale brand new building building ...,amazing view great studio balcony
3,spacethis single room shared bathroom comforta...,single room in palermo soho
4,spacei offer room apartment located olivos dis...,room with river sight
...,...,...
20080,relajate en este alojamiento único tranquilo,hermoso departamento en cañitas
20081,inmejorable zona palier privado hermoso depart...,departamento de categoría
20082,disfrutá de la sencillez de este alojamiento t...,loft parrilla y terraza en ba
20083,hermoso depto de un dormitorio en belgrano ide...,dpto un dormitorio con balcón


DIVIDO EN TESTEO Y ENTRENAMIENTO

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['description'], df['name'], test_size=0.2, random_state=42)

# Crear un objeto vectorizador TF-IDF
tfidf = TfidfVectorizer()

# Ajustar el vectorizador a los datos de entrenamiento
tfidf.fit(X_train)

# Transformar los datos de entrenamiento y prueba en vectores TF-IDF
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Entrenar el modelo SVM
svm_clf = SVC()
svm_clf.fit(X_train_tfidf, y_train)

# Evaluar el modelo en los datos de prueba
y_pred = svm_clf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
X_test

Realizo una prediccion con el conjunto de datos xtest

In [ ]:
# Evaluar el modelo en los datos de prueba
y_pred = svm_clf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.2f}")
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 0.94
Precision: 0.9408512490653835
Recall: 0.9440470107307103
F1 score: 0.9363932942611186


Veo el rendimiento

Guardo el modelo

In [ ]:
!pip install joblib
import joblib as jb
jb.dump(svm_clf, 'svm_clf_room_type2.pkl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


['svm_clf_room_type2.pkl']

El Error que estoy detallando en este caso se dapara cuando use como instancia de clasificacion "room_type". ahora en este momento estoy usando "name" considero que es mejor pero lleva horas entrenando. en cuanto tenga el modelo y la clasificacion lo subo.

Hice una interfaz grafica con TKINTER, esta es simple , y al parecer funciona perfectamente. El problema se da cuando llamo el modelo para precedir un resultado(que probablemente este mal que no hice una interfaz capaz de responder adecuadamente, solo queria probra obtener una respuesta deacurdo a la clasificacion del modelo, luego de ello buscaria mejorar la respuesta de acuerdo a la clasificacion, ya buscando en la base de datos o dataset)

el error que obtengo es: 

  File "C:\Users\enzol\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "c:\Users\enzol\OneDrive\Escritorio\IAchat\appy.py", line 50, in responder
    categoria = clasificar_mensaje(mensaje)
  File "c:\Users\enzol\OneDrive\Escritorio\IAchat\appy.py", line 40, in clasificar_mensaje
    mensaje_vectorizado = tfidf.transform([mensaje_procesado])
  File "C:\Users\enzol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py", line 2155, in transform
    check_is_fitted(self, msg="The TF-IDF vectorizer is not fitted")
  File "C:\Users\enzol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py", line 1390, in check_is_fitted
    raise NotFittedError(msg % {"name": type(estimator).__name__})
sklearn.exceptions.NotFittedError: The TF-IDF vectorizer is not fitted



probé modificando algunas cosas en el modelo, tambien vectorice y trabaje el dato de entrada(osea la consulta) de forma que considero adecuada y obtuve errores como " estas tratando de transformar un objeto string a float"

en este caso el codigo de la interfaz:
from tkinter import scrolledtext, INSERT
from sklearn.feature_extraction.text import TfidfVectorizer
from tkinter import ttk

import joblib as jb




import tkinter as tk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import re

# Cargar el modelo SVM previamente entrenado para la clasificación de room_type
svm_clf_room_type = jb.load('C:/Users/enzol/OneDrive/Escritorio/IAchat/svm_clf_room_type2.pkl')

stop_words = set(stopwords.words('english'))
tfidf = TfidfVectorizer()

def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Quitar puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Quitar números
    text = re.sub('\d+', '', text)
    # Quitar palabras vacías
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Definir la función para clasificar el mensaje de entrada
def clasificar_mensaje(mensaje):
    # Aplicar preprocesamiento al mensaje de entrada
    mensaje_procesado = preprocess_text(mensaje)
    # Vectorizar el mensaje de entrada
    mensaje_vectorizado = tfidf.transform([mensaje_procesado])
    # Clasificar el mensaje utilizando el modelo SVM
    prediccion = svm_clf_room_type.predict(mensaje_vectorizado)
    # Devolver la categoría predicha
    return prediccion[0]

def responder():
    # Obtener el mensaje de entrada del usuario
    mensaje = entrada_texto.get()
    # Clasificar el mensaje utilizando el modelo SVM
    categoria = clasificar_mensaje(mensaje)
    # Devolver la respuesta correspondiente a la categoría predicha
    if categoria == 'pregunta':
        respuesta_texto.set('¿En qué puedo ayudarte?')
    elif categoria == 'comentario_positivo':
        respuesta_texto.set('¡Gracias por tus comentarios!')
    elif categoria == 'comentario_negativo':
        respuesta_texto.set('Lamentamos que no hayas tenido una buena experiencia. ¿Podrías proporcionar más detalles?')
    else:
        respuesta_texto.set('Lo siento, no entiendo tu pregunta. Por favor, inténtalo de nuevo.')